## PyTorch Datasets

In [ ]:
from torchvision import transforms
import sys
import os

sys.path.append(os.path.abspath(".."))
from src.dataset import ImageDataset

transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

root_train = os.path.join('..', 'data', 'ImageNetSubset')
root_test = os.path.join('..', 'data', 'collected_dataset')

dataset_train = ImageDataset(root=root_train, transform=transform, train=True)
dataset_test = ImageDataset(root=root_test, transform=transform, train=False)


## PyTorch Dataloaders

In [ ]:
from torch import Generator
from torch.utils.data import random_split, DataLoader

train_size = int(0.8 * len(dataset_train))
test_size = len(dataset_train) - train_size

dataset_train, dataset_val = random_split(
    dataset_train,
    [train_size, test_size],
    generator=Generator().manual_seed(42),
)

trainloader = DataLoader(dataset_train, batch_size=32, shuffle=True, num_workers=4)
valloader = DataLoader(dataset_val, batch_size=32, shuffle=False, num_workers=4)
testloader = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=4)

## Import Model

In [ ]:
from src.models import SimpleCNN, LargerNet

net = LargerNet()

## Training Loop

In [ ]:
# vorher per pip3 tensorflow installieren

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch import nn, optim 
import torch
import time

net = LargerNet()
early_stopper = EarlyStopper(patience=5, min_delta=0.1)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)
net.to(device)

writer = SummaryWriter(log_dir=f'./runs/LargerNet/exp_{int(time.time())}')

num_epochs = 200

for epoch in range(num_epochs):
    # --- Training ---
    net.train()
    running_loss_sum = 0.0
    train_samples = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        bs = inputs.size(0)
        running_loss_sum += loss.item() * bs
        train_samples += bs

    # --- Validation ---
    net.eval()
    val_loss_sum = 0.0
    val_samples = 0
    correct = 0

    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            bs = inputs.size(0)
            val_loss_sum += loss.item() * bs
            val_samples += bs

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()

    avg_train_loss = running_loss_sum / train_samples
    avg_val_loss = val_loss_sum / val_samples if val_samples > 0 else 0.0
    val_acc = correct / val_samples if val_samples > 0 else 0.0

    # --- TensorBoard Logging ---
    writer.add_scalar('Loss/train_epoch', avg_train_loss, epoch)
    writer.add_scalar('Loss/val_epoch', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/val_epoch', val_acc, epoch)

    print(
        f'Epoch {epoch+1}/{num_epochs} '
        f'- train_loss: {avg_train_loss:.4f} '
        f' val_loss: {avg_val_loss:.4f} '
        f' val_acc: {val_acc:.4f}'
    )

    if early_stopper.early_stop(avg_val_loss):
        print('Stopped\nMin Validation Loss:', early_stopper.min_validation_loss)
        break


print('Finished Training')
writer.close()

## Model Testen

In [ ]:
correct = 0
total = 0

all_predictions = torch.Tensor()
all_labels = torch.Tensor()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predictions = torch.cat((all_predictions, predicted.cpu()))
        all_labels = torch.cat((all_labels, labels.cpu()))

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

## Ergebnisse Speichern

In [ ]:
# ACHTUNG: Dateinamen anpassen! sonst wird altes Modell überschrieben
PATH = 'image_net_2511xx_2.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
!tensorboard --logdir='c:\Users\elias\Downloads\events.out.tfevents.1764768203.db0cf8b52daa.47.3'